## Setup Environments

In [1]:
%run Setup.ipynb

# Exploring Vector Database Operations in LangChain

## Install Chroma Vector DB and LangChain wrapper

In [7]:
!pip install -qq langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.73.1 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.


In [2]:
documents = ['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.']

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [3]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

In [5]:
# delete vector db if exists
!rm -rf ./chroma_db

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [8]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='search_docs',
                   embedding_function=openai_embed_model,
                   persist_directory="./chroma_db")

We take some sample documents

In [9]:
documents

['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.']

We create document IDs to uniquely identify each document

In [10]:
ids = ['doc_'+str(i) for i in range(len(documents))]
ids

['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4']

Checking the Vector DB to see if its empty

In [11]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

### Adding documents to Vector DB

Here we take our texts, pass them through the Open AI embedder to get embeddings and add it to the Chroma Vector DB.

If you have documents in the LangChain `Document` format then you can use `add_documents` instead

In [12]:
chroma_db.add_texts(texts=documents, ids=ids)

['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4']

We check out Vector DB now to see these documents have been indexed successfully

In [13]:
chroma_db.get()

{'ids': ['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None, None, None]}

Run some search queries in our Vector DB

In [14]:
query = 'Tell me about AI'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={}, page_content='Artificial Intelligence aims to create machines that can think and learn.'),
  0.8805678486824036)]

In [15]:
query = 'Do you know about the pyramids?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_4', metadata={}, page_content='The pyramids of Egypt are historical monuments that have stood for thousands of years.'),
  0.8856477737426758)]

In [16]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_1', metadata={}, page_content='Photosynthesis is the process by which green plants make food using sunlight.'),
  1.5097492933273315)]

### Adding more documents to our Vector DB

You can add new documents anytime to the vector DB as shown below

In [17]:
new_documents = [ 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Yoga is an ancient practice that involves physical postures and meditation.']

In [18]:
new_ids = ['doc_'+str(i+len(ids)) for i in range(len(new_documents))]
new_ids

['doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9']

In [19]:
chroma_db.add_texts(texts=new_documents, ids=new_ids)

['doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9']

In [20]:
chroma_db.get()

{'ids': ['doc_0',
  'doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8',
  'doc_9'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
  'Biology is the study of living organisms and their interactions with the environment.',
  'Music therapy can aid in the mental well-being of individuals.',
  'The Milky Way is just one of billions of galaxies in the universe.',
  'Economic theories help understand the distribution of resources in society.',
  'Yoga is an ancient practice that involves physical postures and meditation.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': 

### Do a Similarity Search in the Vector Database After Adding New Documents

Now that we've added more documents to our vector database, let's perform a similarity search again to see how the results may change with the expanded knowledge base.


In [22]:
query = 'What is Biology?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_5', metadata={}, page_content='Biology is the study of living organisms and their interactions with the environment.'),
  0.6787383556365967)]

### Updating documents in the Vector DB

While building toward a real application, you want to go beyond adding data, and also update and delete data.

Chroma has users provide ids to simplify the bookkeeping here and update documents as shown below using the `update_documents`function

In [23]:
chroma_db.get(['doc_3'])

{'ids': ['doc_3'],
 'embeddings': None,
 'documents': ['Artificial Intelligence aims to create machines that can think and learn.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

In [24]:
from langchain_core.documents import Document

ids = ['doc_3']
texts = ['AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.']
documents = [Document(page_content=text, metadata={'doc': id})
                for id, text in zip(ids,texts)]
documents

[Document(metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.')]

In [25]:
chroma_db.update_documents(ids=ids,documents=documents)

In [26]:
chroma_db.get(['doc_3'])

{'ids': ['doc_3'],
 'embeddings': None,
 'documents': ['AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'doc': 'doc_3'}]}

In [27]:
query = 'What is AI?'
docs = chroma_db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'),
  0.5633718967437744)]

### Deleting documents in the Vector DB

Chroma has users provide ids to simplify the bookkeeping here and delete documents as shown below using the `delete`function

In [28]:
chroma_db.delete(['doc_9'])

In [29]:
chroma_db.get()

{'ids': ['doc_0',
  'doc_1',
  'doc_2',
  'doc_3',
  'doc_4',
  'doc_5',
  'doc_6',
  'doc_7',
  'doc_8'],
 'embeddings': None,
 'documents': ['Quantum mechanics describes the behavior of very small particles.',
  'Photosynthesis is the process by which green plants make food using sunlight.',
  "Shakespeare's plays are a testament to English literature.",
  'AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.',
  'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
  'Biology is the study of living organisms and their interactions with the environment.',
  'Music therapy can aid in the mental well-being of individuals.',
  'The Milky Way is just one of billions of galaxies in the universe.',
  'Economic theories help understand the distribution of resources in society.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None,
  None,
  None,
  {'doc':

### Load Vector DB from disk

Once you have saved your DB to disk, you can load it up anytime and connect to it and run queries as shown below

In [30]:
# load from disk
db = Chroma(persist_directory="./chroma_db",
            embedding_function=openai_embed_model,
            collection_name='search_docs')

query = 'What is AI?'
docs = db.similarity_search_with_score(query=query, k=1)
docs

[(Document(id='doc_3', metadata={'doc': 'doc_3'}, page_content='AI is known as Artificial Intelligence. Artificial Intelligence aims to create machines that can think and learn.'),
  0.5634348392486572)]